In [1]:
import shnitsel as sh
import shnitsel.xarray
import xarray as xr
from IPython.display import display, Image

In [2]:
A01 = sh.open_frames('/nc/reports/2025-05-21_datasheets/filtered_C2H4.nc')
I01 = sh.open_frames('/nc/SHNITSEL_databases/dynamic/I01_ch2nh2_dynamic.nc')
A02 = sh.open_frames('/nc/reports/2025-05-21_datasheets/filtered_C3H6.nc')
A03 = sh.open_frames('/nc/reports/2025-05-21_datasheets/filtered_C4H8_g0.nc')

In [3]:
bl = A01.atXYZ.sh.get_bond_lengths()
bl

<xarray.DataArray (frame: 62031, bond: 5)> Size: 2MB
array([[1.34517946, 1.13050448, 1.05540845, 0.93322882, 1.1162366 ],
       [1.35167321, 1.11766826, 1.044994  , 0.95831056, 1.11812979],
       [1.35899658, 1.10153534, 1.03910828, 1.00143008, 1.11657425],
       ...,
       [7.10717786, 1.29207555, 1.1900943 , 1.08906804, 1.14459848],
       [7.12481371, 1.27515023, 1.18119096, 1.08619924, 1.1605664 ],
       [7.14361786, 1.25142808, 1.16852549, 1.08623084, 1.17408787]],
      shape=(62031, 5))
Coordinates:
  * frame      (frame) object 496kB MultiIndex
  * trajid     (frame) int64 496kB 2 2 2 2 2 2 2 ... 962 962 962 962 962 962 962
  * time       (frame) float64 496kB 0.0 0.5 1.0 1.5 ... 167.0 167.5 168.0 168.5
  * bond       (bond) object 40B MultiIndex
  * bond_type  (bond) object 40B 'CC' 'HC' 'HC' 'HC' 'HC'
  * atom1      (bond) int64 40B 0 0 0 1 1
  * atom2      (bond) int64 40B 1 2 5 3 4

In [9]:
limits = bl.sh.msel(HC=1, CC=2)
limits

<xarray.DataArray (bond: 5)> Size: 40B
array([2, 1, 1, 1, 1])
Coordinates:
  * bond       (bond) object 40B MultiIndex
  * bond_type  (bond) <U2 40B 'CC' 'HC' 'HC' 'HC' 'HC'
  * atom1      (bond) int64 40B 0 0 0 1 1
  * atom2      (bond) int64 40B 1 2 5 3 4

In [10]:
masks = bl < limits
masks

<xarray.DataArray (frame: 62031, bond: 5)> Size: 310kB
array([[ True, False, False,  True, False],
       [ True, False, False,  True, False],
       [ True, False, False, False, False],
       ...,
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False]], shape=(62031, 5))
Coordinates:
  * frame      (frame) object 496kB MultiIndex
  * trajid     (frame) int64 496kB 2 2 2 2 2 2 2 ... 962 962 962 962 962 962 962
  * time       (frame) float64 496kB 0.0 0.5 1.0 1.5 ... 167.0 167.5 168.0 168.5
  * bond       (bond) object 40B MultiIndex
  * bond_type  (bond) object 40B 'CC' 'HC' 'HC' 'HC' 'HC'
  * atom1      (bond) int64 40B 0 0 0 1 1
  * atom2      (bond) int64 40B 1 2 5 3 4

In [5]:
cutoffs = masks.sh.last_time_where()
cutoffs

<xarray.DataArray 'time' (trajid: 297, bond: 5)> Size: 12kB
array([[ 83.5,  -1. ,  -1. ,   0.5,  -1. ],
       [109. ,  -1. ,  -1. ,  -1. ,   0. ],
       [ 94. ,  -1. ,  -1. ,  -1. ,  -1. ],
       ...,
       [ 72.5,  -1. ,  -1. ,  -1. ,  -1. ],
       [ 81. ,  -1. ,  -1. ,  -1. ,  -1. ],
       [ 17.5,  -1. ,  -1. ,  -1. ,  -1. ]], shape=(297, 5))
Coordinates:
  * trajid     (trajid) int64 2kB 2 3 4 8 10 11 15 ... 950 951 956 957 961 962
  * bond       (bond) object 40B MultiIndex
  * bond_type  (bond) object 40B 'CC' 'HC' 'HC' 'HC' 'HC'
  * atom1      (bond) int64 40B 0 0 0 1 1
  * atom2      (bond) int64 40B 1 2 5 3 4

In [6]:
limits = dict(CC=1.5, HC=2, CN=3, HN=2)
cpnds = {'I01': I01, 'A01': A01, 'A02': A02, 'A03': A03}
cutoffs = {}
masks = {}
for cpnd in cpnds:
    lengths = cpnds[cpnd].atXYZ.sh.get_bond_lengths()
    masks[cpnd] = lengths < lengths.sh.msel(**limits)
    cutoffs[cpnd] = masks[cpnd].sh.last_time_where().rename(bond='cutoff')
    cutoffs[cpnd]['original'] = lengths.time.groupby('trajid').last()
    cutoffs[cpnd]['earliest'] = cutoffs[cpnd].min('cutoff')

In [7]:
cutoffs['A01'] #.rename(bond='cutoff')

<xarray.DataArray 'time' (trajid: 297, cutoff: 5)> Size: 12kB
array([[ 44.5,  83.5,  83.5,  83.5,  83.5],
       [ 26.5,  82.5, 109. , 109. ,  40. ],
       [ 26. ,  94. ,  94. ,  72.5,  94. ],
       ...,
       [  8. ,  72.5,  40.5,  72.5,  72.5],
       [ 23. ,  81. ,  81. ,  81. ,  81. ],
       [  4.5, 168.5, 168.5, 168.5, 168.5]], shape=(297, 5))
Coordinates:
  * trajid     (trajid) int64 2kB 2 3 4 8 10 11 15 ... 950 951 956 957 961 962
  * cutoff     (cutoff) object 40B MultiIndex
  * bond_type  (cutoff) object 40B 'CC' 'HC' 'HC' 'HC' 'HC'
  * atom1      (cutoff) int64 40B 0 0 0 1 1
  * atom2      (cutoff) int64 40B 1 2 5 3 4
    original   (trajid) float64 2kB 83.5 109.0 94.0 199.5 ... 72.5 81.0 168.5
    earliest   (trajid) float64 2kB 44.5 26.5 26.0 28.0 4.5 ... 9.0 8.0 23.0 4.5

## Which bond breaks first, how often?

In [ ]:
import numpy as np

co = cutoffs['A03']
nreasons = co.sizes['cutoff']
typefreqs = np.bincount(co.argmin('cutoff'), minlength=nreasons)
typefreqs

In [101]:
display_data = co['cutoff'].copy(data=typefreqs)
display_data = 100*display_data / display_data.sum()

In [ ]:
mol = A03.sh.default_mol()

def label_bonds(mol, da):
    for b in mol.GetBonds():
        ai1 = b.GetBeginAtom().GetIdx()
        ai2 = b.GetEndAtom().GetIdx()
        datum = da.sel(atom1=ai1, atom2=ai2).item()
        if datum != 0:
            b.SetProp("bondNote", f"{datum:.3}%")
    return mol

label_bonds(mol, display_data)

In [ ]:
png = sh.dynamic.datasheet.plot.

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2, 2)
sh.dynamic.pca_biplot.mpl_imshow_png(axs[0,0])

In [ ]:
for (cn, co) in cutoffs.items():
    nreasons = co.sizes['cutoff']
    typefreqs = np.bincount(co.argmin('cutoff'), minlength=nreasons)
    display_data = co['cutoff'].copy(data=typefreqs)
    display_data = 100*display_data / display_data.sum()
    mol = cpnds[cn].sh.default_mol()
    display(label_bonds(mol, display_data))

## Which bonds break at all, and how often?

In [ ]:
masks['A03'].groupby('trajid').all('frame').sum('trajid')

In [ ]:
for (cn, mask) in masks.items():
    display_data = (~mask.groupby('trajid').all('frame')).sum('trajid')
    display_data = 100*display_data / display_data.sum()
    mol = cpnds[cn].sh.default_mol()
    display(label_bonds(mol, display_data))


## Other plot, maybe leave out

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(2, 4, sharey=False, constrained_layout=True, height_ratios=[1, 3])
for (cn, c), axcol in zip(cutoffs.items(), axs.T):
    nreasons = c.sizes['cutoff']
    typefreqs = np.bincount(c.argmin('cutoff'), minlength=nreasons)
    xticks = range(nreasons)
    axcol[0].bar(xticks, typefreqs)
    axcol[0].set_xticks(xticks)
    axcol[0].set_xticklabels(labels=c['bond_type'].data, rotation=90, ha='center', va='center')
    axcol[0].set_title(cn)

    c['most_frequent'] = 'trajid', c.isel(cutoff=typefreqs.argmax()).data
    c = c.sortby(['earliest', 'most_frequent'])
    xticks = np.arange(c.sizes['trajid'])
    for ctn, ctv in c.groupby('cutoff'):
        ctv = ctv.squeeze()
        axcol[1].barh(xticks, ctv, height=1.0, alpha=0.2)
    
    axs[0,0].set_ylabel("# trajs truncated\nfor given reason")
    axs[1,0].set_ylabel("$t$ / fs")
    for ax in axs[1, :]:
        ax.set_xlabel("# trajs with this cutoff")

In [ ]:
A01.sh.save_frames('/tmp/I01_intact.nc')
I01.sh.save_frames
A02.sh.save_frames
A03.sh.save_frames

In [ ]:
# TODO
# Original: use substructures
# Filter cleavages
# A01.atXYZ.attrs['smiles_map'] = A01.atXYZ.isel(frame=0).sh.smiles_map()
# I01.atXYZ.attrs['smiles_map'] = I01.atXYZ.isel(frame=0).sh.smiles_map(charge=+1)
# A02new.atXYZ.attrs['smiles_map'] = A02new.atXYZ.isel(frame=0).sh.smiles_map(charge=-3)
# A03new.atXYZ.attrs['smiles_map'] = A03new.atXYZ.isel(frame=0).sh.smiles_map(charge=-8)
# filtration_verbosity = 0
# I01 = sh.dynamic.filter_unphysical.filter_cleavage(I01, CH=True, CN=True, NH=True, verbose=filtration_verbosity)
# A01 = sh.dynamic.filter_unphysical.filter_cleavage(A01, CC=True, CH=True, verbose=filtration_verbosity)
# A02new = sh.dynamic.filter_unphysical.filter_cleavage(A02new, CC=True, CH=True, verbose=filtration_verbosity)
# A03new = sh.dynamic.filter_unphysical.filter_cleavage(A03new, CC=True, CH=True, verbose=filtration_verbosity)